In [1]:
import sqlite3
import pandas as pd
import folium
import math

In [2]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')

cursor = conn.cursor()

In [4]:
df_loc

,latitude_copernicus,longitude_copernicus,latitude_embalses,longitude_embalses,latitude_aemet,longitude_aemet,latitude_rios_canales,longitude_rios_canales,nombre_aemet,estacion_aforo_rios_canales,embalse
0,39.174,-1.255,39.1435,-1.0200,39.208333,-1.094444,39.196752,-1.425311,JALANCE,ALCALA DEL JUCAR,Embarcaderos
1,39.174,-1.255,39.1435,-1.0200,39.082778,-1.100833,39.196752,-1.425311,ZARRA,ALCALA DEL JUCAR,Embarcaderos
2,39.174,-1.255,39.1233,-1.1427,39.208333,-1.094444,39.196752,-1.425311,JALANCE,ALCALA DEL JUCAR,"Molinar, El"
3,39.174,-1.255,39.1233,-1.1427,39.082778,-1.100833,39.196752,-1.425311,ZARRA,ALCALA DEL JUCAR,"Molinar, El"
4,39.174,-1.255,39.1435,-1.0200,39.208333,-1.094444,39.339757,-1.341927,JALANCE,VILLATOYA,Embarcaderos
5,39.174,-1.255,39.1435,-1.0200,39.082778,-1.100833,39.339757,-1.341927,ZARRA,VILLATOYA,Embarcaderos
6,39.174,-1.255,39.1233,-1.1427,39.208333,-1.094444,39.339757,-1.341927,JALANCE,VILLATOYA,"Molinar, El"
7,39.174,-1.255,39.1233,-1.1427,39.082778,-1.100833,39.339757,-1.341927,ZARRA,VILLATOYA,"Molinar, El"
8,39.174,-1.255,39.1435,-1.0200,39.208333,-1.094444,39.363772,-1.443212,JALANCE,LA TERRERA,Embarcaderos
9,39.174,-1.255,39.1435,-1.0200,39.082778,-1.100833,39.363772,-1.443212,ZARRA,LA TERRERA,Embarcaderos


In [6]:
pixel = 190

query = f'''
    SELECT 
        loc_copernicus.latitude AS latitude_copernicus,
        loc_copernicus.longitude AS longitude_copernicus,
        loc_embalses.latitude AS latitude_embalses,
        loc_embalses.longitude AS longitude_embalses,
        loc_aemet.latitude AS latitude_aemet,
        loc_aemet.longitude AS longitude_aemet,
        loc_rios_canales.latitude AS latitude_rios_canales,
        loc_rios_canales.longitude AS longitude_rios_canales,
        -- Información adicional
        aemet_info.nombre AS nombre_aemet,
        rios_canales_info.EstacióndeAforo AS estacion_aforo_rios_canales,
        embalses_info.Embalse AS embalse
    FROM df_pixeles_cercanos p
    -- Relacionar las localizaciones de la tabla df_pixeles_cercanos con las latitudes y longitudes
    INNER JOIN locations_id loc_copernicus ON loc_copernicus.location_id = p.location_id_copernicus
    INNER JOIN locations_id loc_embalses ON loc_embalses.location_id = p.location_id_embalses
    INNER JOIN locations_id loc_aemet ON loc_aemet.location_id = p.location_id_aemet
    INNER JOIN locations_id loc_rios_canales ON loc_rios_canales.location_id = p.location_id_rios_canales
    -- Relacionar con tablas de información adicional
    INNER JOIN df_aemet_info aemet_info ON aemet_info.location_id = loc_aemet.location_id
    INNER JOIN df_rios_canales_info rios_canales_info ON rios_canales_info.location_id = loc_rios_canales.location_id
    INNER JOIN df_embalses_info embalses_info ON embalses_info.location_id = loc_embalses.location_id
    -- Filtrar por location_id_copernicus = {pixel}
    WHERE p.location_id_copernicus = {pixel};
        '''
# Ejecutar la consulta
cursor.execute(query)


df_loc = pd.read_sql_query(query, conn)

# Coordenadas base para centrar el mapa
map_center = [39.0, -0.5]  # Aproximadamente en la cuenca del Júcar

# Crear el mapa base
m = folium.Map(location=map_center, zoom_start=9.5)

# Añadir el cuadrado de la zona de Copernicus (representando un área más grande)
for index, row in df_loc.iterrows():
    # Longitud y latitud del centro de Copernicus
    lat_copernicus = row['latitude_copernicus']
    lon_copernicus = row['longitude_copernicus']

    # Definir un lado del cuadrado basado en 0.5 grados
    size_lat = 0.5  # 0.5 grados de latitud
    size_lon = 0.5 * 111.32 * math.cos(math.radians(lat_copernicus)) / 111.32  # Ajustado por la latitud (en grados)

    additional_lon = 5 / (111.32 * math.cos(math.radians(lat_copernicus))) 
    # Crear las coordenadas de las esquinas del cuadrado
    bounds = [
        [lat_copernicus - size_lat / 2, lon_copernicus - size_lon / 2],  # Esquina inferior izquierda
        [lat_copernicus + size_lat / 2, lon_copernicus + size_lon / 2 + additional_lon]   # Esquina superior derecha
    ]
    
    # Añadir el cuadrado de Copernicus al mapa
    folium.Rectangle(
        bounds=bounds,
        color='blue',
        weight=2,
        fill=True,
        fill_color='blue',
        fill_opacity=0,
        popup='Copernicus Area'
    ).add_to(m)

    # Añadir marcador para Aemet
    folium.Marker(
        location=[row['latitude_aemet'], row['longitude_aemet']],
        popup=f"Aemet: {row['nombre_aemet']}",
        icon=folium.Icon(color='red')
    ).add_to(m)

    # Añadir marcador para Ríos Canales
    folium.Marker(
        location=[row['latitude_rios_canales'], row['longitude_rios_canales']],
        popup= f"Ríos Canales: {row['estacion_aforo_rios_canales']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    # Añadir marcador para Ríos Canales
    folium.Marker(
        location=[row['latitude_embalses'], row['longitude_embalses']],
        popup=f"Embalse: {row['embalse']}",
        icon=folium.Icon(color='orange')
    ).add_to(m)

# Mostrar el mapa
m

In [77]:
query = '''
        SELECT 
            d.date,
            c.total_precipitation,
            c.skin_temperature,
            c.evaporation,
            c.runoff,
            c.snowfall,
            c.soil_water_l1,
            c.soil_water_l2,
            c.soil_water_l3,
            c.soil_water_l4,
            c.high_vegetation_cover,
            c.low_vegetation_cover 
            FROM df_copernicus c JOIN df_date d ON d.date_id = c.date_id  WHERE c.location_id = 190;
        '''
# Ejecutar la consulta
cursor.execute(query)


df_c = pd.read_sql_query(query, conn)

In [87]:
query = '''
        SELECT 
            d.date,
            aem.tmed,
            aem.prec,
            aem.location_id
            FROM df_aemet aem 
            JOIN df_date d ON d.date_id = aem.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_aemet = aem.location_id 
            WHERE pc.location_id_copernicus = 190;
        '''
# Ejecutar la consulta
cursor.execute(query)


df_aemet = pd.read_sql_query(query, conn)

In [92]:
query = '''
        SELECT 
            d.date,
            rios.quantity_hm3,
            rios.location_id
            FROM df_rios_canales rios 
            JOIN df_date d ON d.date_id = rios.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_rios_canales = rios.location_id 
            WHERE pc.location_id_copernicus = 190;
        '''
# Ejecutar la consulta
cursor.execute(query)


df_rios = pd.read_sql_query(query, conn)

In [94]:
query = '''
        SELECT 
            d.date,
            emb.quantity_hm3,
            emb.location_id
            FROM df_embalses emb 
            JOIN df_date d ON d.date_id = emb.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_embalses = emb.location_id 
            WHERE pc.location_id_copernicus = 190;
        '''
# Ejecutar la consulta
cursor.execute(query)


df_embalses = pd.read_sql_query(query, conn)

In [ ]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')
cursor = conn.cursor()

query = '''
SELECT 
    d.date AS fecha,
    c.total_precipitation,
    c.skin_temperature,
    c.evaporation,
    c.runoff,
    c.snowfall,
    c.soil_water_l1,
    c.soil_water_l2,
    c.soil_water_l3,
    c.soil_water_l4,
    c.high_vegetation_cover,
    c.low_vegetation_cover,
    e.quantity_hm3 AS quantity_hm3_embalses,
    a.tmed,
    a.prec,
    a.tmin,
    a.tmax,
    a.dir,
    a.velmedia,
    a.racha,
    r.quantity_hm3 AS quantity_hm3_rios_canales
FROM df_pixeles_cercanos p
-- Subconsulta para obtener los location_ids relacionados con location_id_copernicus = 14
INNER JOIN (
    SELECT location_id_embalses, location_id_aemet, location_id_rios_canales
    FROM df_pixeles_cercanos
    WHERE location_id_copernicus = 14
) rel ON p.location_id_embalses = rel.location_id_embalses
        AND p.location_id_aemet = rel.location_id_aemet
        AND p.location_id_rios_canales = rel.location_id_rios_canales
-- Unir las tablas con las series temporales correspondientes
LEFT JOIN df_embalses e ON e.location_id = p.location_id_embalses
LEFT JOIN df_aemet a ON a.location_id = p.location_id_aemet
LEFT JOIN df_rios_canales r ON r.location_id = p.location_id_rios_canales
LEFT JOIN df_copernicus c ON c.location_id = p.location_id_copernicus
LEFT JOIN df_date d ON d.date_id = c.date_id
WHERE p.location_id_copernicus = 14
ORDER BY d.date;
'''

# Ejecutar la consulta
cursor.execute(query)


df = pd.read_sql_query(query, conn)